In [1]:
# PySMT - for SAT solving
import pysmt.shortcuts as pysmtsol # Symbol, And, Not, Xor, is_sat


In [2]:
# PyEDA - for circuit benchmark creation and cnf generation 
import pyeda
import pyeda.boolalg.expr as eda # exprvar, Or, And, Xor, Not, expr2dimacscnf

In [3]:
from pysmt.smtlib.parser import open_
from pysmt.shortcuts import FreshSymbol, Symbol


def read_dimacs(fname):
    """Read a DIMACS CNF file from the given file.

    Returns a tuple: (vars_cnt, clauses, comments)
    """
    prob_type, vars_cnt, clauses_cnt = None, None, None
    max_var = 0
    comments = []
    clauses = []

    with open_(fname) as fin:
        for line in fin:
            if line[0] == "c":
                comments.append(line)
            elif line[0] == "p":
                _, prob_type, vars_cnt, clauses_cnt = line.split(" ")
                prob_type = prob_type.strip()
                if prob_type != "cnf":
                    raise IOError("File does not contain a cnf.")
                vars_cnt = int(vars_cnt.strip())
                clauses_cnt = int(clauses_cnt)
                break

        for line in fin:
            if line[0] == "c":
                comments.append(line)
            else:
                # TODO: More robust parsing of clauses
                cl = line.strip().split(" ")
                assert cl[-1].strip() == "0", cl
                clause = [int(lit) for lit in cl[:-1]]
                max_var = max(max_var, max(abs(lit) for lit in clause))
                assert not any(lit == 0 for lit in clause), clause
                clauses.append(clause)

    # Validation
    if clauses_cnt != len(clauses):
        raise IOError("Mismatch between declared clauses (%d) " % clauses_cnt +
                      "and actual clauses (%d) in DIMACS file." % len(clauses))
    if max_var > vars_cnt:
        raise IOError("Mismatch between declared variables (%d) " % vars_cnt +
                      "and actual variables (%d) in DIMACS file." % max_var)

    return vars_cnt, clauses, comments


def dimacs_to_pysmt(vars_cnt, clauses, comments):
    """Convert a DIMACS structure into a pySMT formula.

    Returns (formula, symbol_table). The symbol_table contains a
    mapping from pySMT symbol to DIMACS var_idx.

    """
    st = {}
    rev_st = {}
    for i in range(1, vars_cnt+1):
        # s = FreshSymbol(template=("_dimacs_%d"%i))
        s = Symbol(name=("_dimacs_%d"%i))
        st[i] = s
        st[-i] = pysmtsol.Not(s)
        rev_st[s] = i
    res = pysmtsol.And(pysmtsol.Or(st[lit] for lit in clause) \
              for clause in clauses)
    return res, rev_st

In [4]:
def read_dimacs_bench(circuit, benchname):
    vars_cnt, clauses, comments = read_dimacs(f'cnf_bench/{circuit}/{benchname}.cnf')
    cir, rev_st = dimacs_to_pysmt(vars_cnt, clauses, comments)
    # print("read dimacs :", cir)
    # print("symbol table: ", rev_st)
    return cir

In [5]:
def create_miter(impl_cir, spec_cir):
    miter = pysmtsol.Xor(impl_cir, spec_cir)
    return miter

In [6]:
def check_sat(miter):
    sat_res = pysmtsol.is_sat(miter)
    return sat_res

In [7]:
def create_intermediate_miters(circuit, out1, out2):
    impl_cir = read_dimacs_bench(circuit, out1)
    spec_cir = read_dimacs_bench(circuit, out2)
    miter_cir = create_miter(impl_cir, spec_cir)
    return miter_cir

    
def equivalence_check(circuit, bench1, bench2, out):
    # print("bench1 : ", bench1, "  bench2 :", bench2)
    miter_lst = [create_intermediate_miters(circuit, bench1[i], bench2[i]) for i in range(out)]
    # print("miter_out: ", miter_lst)
    miter_or = pysmtsol.Or(miter_lst)    

    # print("miter : ", miter_or)

    sat_res = check_sat(miter_or)
    # print("sat_res: ", sat_res)
    if sat_res: print(f"{bench1} and {bench2} are SAT --> UNEQUAL CIRCUITS")
    else: print(f"{bench1} and {bench2} are UNSAT --> EQUIVALENT CIRCUITS")
    return sat_res

In [8]:
circuit = "fullAdder"
circuit1 = ["sum1"]
circuit2 = ["cout2"]
out = 1
res = equivalence_check(circuit, circuit1, circuit2, out)
 


['sum1'] and ['cout2'] are SAT --> UNEQUAL CIRCUITS


In [9]:
def circuit2dimacs(bool_formula):
    ex = bool_formula.to_cnf()
    exp, dimacs_bench = eda.expr2dimacscnf(ex)
    # print("dimacs : ", dimacs_bench)
    return dimacs_bench

def write_dimacs(dimacs_bench, circuit, benchname):
    f = open(f"cnf_bench/{circuit}/{benchname}.cnf", "w")
    f.write(str(dimacs_bench))
    f.close()

def write_circuit(bool_formulae, circuit, benchnames):
    for i,formula in enumerate(bool_formulae):
        dimacs_ = circuit2dimacs(formula)
        write_dimacs(dimacs_, circuit, benchnames[i])
        


# Test - AND gate and its NAND gate equivalent

In [10]:
## AND and NAND
circuit = "AND-NAND"
a, b = map(eda.exprvar, "a b".split())

# AND Circuit 1: 
and1 = eda.And('a', 'b')

# NAND Circuit 2: 
nand2 = eda.Nand(eda.Nand('a','b'), eda.Nand('a','b'))

write_circuit([and1, nand2], circuit, ["and1","nand2"])

## Equivalence check circuit 1 and 2
circuit1 = ["and1"]
circuit2 = ["nand2"]
out = 1
res = equivalence_check(circuit, circuit1, circuit2, out)

['and1'] and ['nand2'] are UNSAT --> EQUIVALENT CIRCUITS


# Multi-AND

In [11]:
## Multi-AND
circuit = "Multi-AND"
a, b, c, d = map(eda.exprvar, "a b c d".split())

# Multi-AND Circuit 1: 
mand1 = eda.And(eda.And(eda.And('a', 'b'), 'c'), 'd')
# mand1 = eda.And('a', 'b')

# Multi-AND Circuit 2:
mand2 = eda.And(eda.And('a','b'), eda.And('c','d')) 

# Multi-AND Circuit 3: 
mand3 = eda.Nand(eda.Nand('a','b'), eda.Nand('c','d'))

# Multi-AND Circuit 4: 
and14 = eda.Nand(eda.Nand('a','b'), eda.Nand('a','b'))
and24 = eda.Nand(eda.Nand('c','d'), eda.Nand('c','d'))
mand4 = eda.Nand(eda.Nand(and14, and24), eda.Nand(and14, and24))


write_circuit([mand1, mand2, mand3, mand4], circuit, ["mand1","mand2","mand3","mand4"])

## Equivalence check circuit 1 and 2
circuit1 = ["mand1"]
circuit2 = ["mand2"]
out = 1
res = equivalence_check(circuit, circuit1, circuit2, out)

## Equivalence check circuit 2 and 3
circuit1 = ["mand2"]
circuit2 = ["mand3"]
out = 1
res = equivalence_check(circuit, circuit1, circuit2, out)

## Equivalence check circuit 3 and 1
circuit1 = ["mand1"]
circuit2 = ["mand3"]
out = 1
res = equivalence_check(circuit, circuit1, circuit2, out)

## Equivalence check circuit 4 and 1
circuit1 = ["mand1"]
circuit2 = ["mand4"]
out = 1
res = equivalence_check(circuit, circuit1, circuit2, out)

['mand1'] and ['mand2'] are UNSAT --> EQUIVALENT CIRCUITS
['mand2'] and ['mand3'] are SAT --> UNEQUAL CIRCUITS
['mand1'] and ['mand3'] are SAT --> UNEQUAL CIRCUITS
['mand1'] and ['mand4'] are UNSAT --> EQUIVALENT CIRCUITS


# Multi-gate

In [12]:
## Multi-gate
circuit = "Multi-gate"
a, b, c, d = map(eda.exprvar, "a b c d".split())

# Multi-gate Circuit 1: 
and11 = eda.And('c','d')
mgate1 = eda.Or(eda.And('a', and11), eda.And('b',eda.Not(and11)))

# Multi-gate Circuit 2:
mgate2 = eda.Xor(eda.And(eda.Xor('a','b'), eda.And('c','d')), 'b') 

# Multi-gate Circuit 3:
mgate3 = eda.Xor(eda.And(eda.Or('a','b'), eda.And('c','d')), 'b')


write_circuit([mgate1, mgate2, mgate3], circuit, ["mgate1","mgate2","mgate3"])

## Equivalence check circuit 1 and 2
circuit1 = ["mgate1"]
circuit2 = ["mgate2"]
out = 1
res = equivalence_check(circuit, circuit1, circuit2, out)

## Equivalence check circuit 1 and 3
circuit1 = ["mgate1"]
circuit2 = ["mgate3"]
out = 1
res = equivalence_check(circuit, circuit1, circuit2, out)


['mgate1'] and ['mgate2'] are UNSAT --> EQUIVALENT CIRCUITS
['mgate1'] and ['mgate3'] are SAT --> UNEQUAL CIRCUITS


# Multi-output circuits

In [17]:
## Multi-out
circuit = "Multi-out"
a, b, c, d = map(eda.exprvar, "a b c d".split())

# Multi-out Circuit 1: 
nand11 = eda.Nand('b','c')
nand21 = eda.Nand('c','d')
nand31 = eda.Nand(nand11, nand21)
mout1_1 = eda.Nand('a',nand31)
mout1_2 = nand21

# Multi-out Circuit 2:
inter2 = eda.Or(eda.And('b','c'),eda.And('c','d'))
mout2_1 = eda.Nand('a', inter2)
mout2_2 = eda.Nand(inter2, 'd') 

# Multi-out Circuit 3:
mout3_1 = eda.And(eda.And('a','b'), eda.And('c','d'))
mout3_2 = eda.Or(eda.Or('b','c'), eda.And('c','d'))

# Multi-out Circuit 4:
mout4_1 = eda.Not(eda.And('c','d'))
mout4_2 = eda.Or(eda.Or('b','c'), eda.And('c','d'))

write_circuit([mout1_1, mout1_2, mout2_1, mout2_2], circuit, ["mout1_1","mout1_2", "mout2_1","mout2_2"])
write_circuit([mout3_1, mout3_2, mout4_1, mout4_2], circuit, ["mout3_1","mout3_2", "mout4_1","mout4_2"])

## Equivalence check circuit 1 and 2
circuit1 = ["mout1_1","mout1_2"]
circuit2 = ["mout2_1","mout2_2"]
out = 2
res = equivalence_check(circuit, circuit1, circuit2, out)

## Equivalence check circuit 3 and 4
circuit1 = ["mout3_1","mout3_2"]
circuit2 = ["mout4_1","mout4_2"]
out = 2
res = equivalence_check(circuit, circuit1, circuit2, out)

['mout1_1', 'mout1_2'] and ['mout2_1', 'mout2_2'] are UNSAT --> EQUIVALENT CIRCUITS
['mout3_1', 'mout3_2'] and ['mout4_1', 'mout4_2'] are SAT --> UNEQUAL CIRCUITS


# Full Adder

In [15]:
## Adder
circuit = "fullAdder"
a, b, ci = map(eda.exprvar, "a b ci".split())

# Adder Circuit 1: 
sum1 =  eda.Xor('a', 'b', 'ci')
cout1 = eda.Or(eda.And('a', 'b'), eda.And('a', 'ci'), eda.And('b', 'ci'))

# Adder Circuit 2: 
sum2 = ~a & ~b & ci | ~a & b & ~ci | a & ~b & ~ci | a & b & ci
cout2 = a & b | a & ci | b & ci | a & b & ci

# Adder Circuit 3: 
sum3 = ~a & ~b & ci | ~a & b & ~ci | a & ~b & ~ci | a & b & ci
cout3 = a & b | a & ci | b & ci | a | b | ci

write_circuit([sum1, cout1, sum2, cout2, sum3, cout3], circuit, ["sum1","cout1","sum2","cout2","sum3","cout3"])

## Equivalence check adder 1 and 2
circuit1 = ["sum1","cout1"]
circuit2 = ["sum2", "cout2"]
out = 2
res = equivalence_check(circuit, circuit1, circuit2, out)

## Equivalence check adder 2 and 3
circuit1 = ["sum2","cout2"]
circuit2 = ["sum3", "cout3"]
out = 2
res = equivalence_check(circuit, circuit1, circuit2, out)

## Equivalence check adder 3 and 1
circuit1 = ["sum1","cout1"]
circuit2 = ["sum3", "cout3"]
out = 2
res = equivalence_check(circuit, circuit1, circuit2, out)

['sum1', 'cout1'] and ['sum2', 'cout2'] are UNSAT --> EQUIVALENT CIRCUITS
['sum2', 'cout2'] and ['sum3', 'cout3'] are SAT --> UNEQUAL CIRCUITS
['sum1', 'cout1'] and ['sum3', 'cout3'] are SAT --> UNEQUAL CIRCUITS


# 2-bit Multiplier

In [24]:
# 2-bit Multiplier
## Multiplier
circuit = "Multiplier"
a0, b0, a1, b1 = map(eda.exprvar, "a0 b0 a1 b1".split())

# Mutliplier Circuit 1: 
and11 = eda.And('a0','b1')
and21 = eda.And('b0', 'a1')
and31 = eda.And('a1','b1')

out0_1 = eda.And('a0', 'b0')
out1_1 = eda.Xor(and11, and21)
out2_1 = eda.Xor(and31, eda.And(and11, and21))
out3_1 = eda.And(and31, eda.And(and11, and21))

write_circuit([out0_1, out1_1, out2_1, out3_1], circuit, ['out0_1', 'out1_1', 'out2_1', 'out3_1'])

# Multiplier Circuit 2: 
def halfadder(a, b):
    sum_ =  eda.Xor(a,b)
    carry_ = eda.And(a,b)
    return sum_, carry_

and12 = eda.And('a0','b1')
and22 = eda.And('b0','a1')
and32 = eda.And('a1','b1')

out0_2 = eda.And('a0', 'b0')
out1_2, carry_ = halfadder(and12, and22)
out2_2, out3_2 = halfadder(carry_, and32)
# out1_2, carry_ = eda.Xor(and12, and22), eda.And(and12, and22)
# out2_2, out3_2 = eda.Xor(carry_, and32), eda.And(carry_, and32)


write_circuit([out0_2, out1_2, out2_2, out3_2], circuit, ['out0_2', 'out1_2', 'out2_2', 'out3_2'])

## Equivalence check adder 1 and 2
circuit1 = ['out0_1', 'out1_1', 'out2_1', 'out3_1']
circuit2 = ['out0_2', 'out1_2', 'out2_2', 'out3_2']
out = 4
res = equivalence_check(circuit, circuit1, circuit2, out)

['out0_1', 'out1_1', 'out2_1', 'out3_1'] and ['out0_2', 'out1_2', 'out2_2', 'out3_2'] are UNSAT --> EQUIVALENT CIRCUITS


# 4-bit Equality Comparator

In [27]:
# Comparator
circuit = "Comparator"
a0, a1, a2, a3, b0, b1, b2, b3 = map(eda.exprvar, "a0 a1 a2 a3 b0 b1 b2 b3".split())

# Comparator Circuit 1: 
comp1 = eda.And(eda.Xnor('a0','b0'), eda.Xnor('a1','b1'), eda.Xnor('a2','b2'), eda.Xnor('a3','b3'))


# Comparator Circuit 2: 
comp2 = eda.Nor(eda.Or(eda.Xor('a0','b0'),eda.Xor('a1','b1')), eda.Or(eda.Xor('a2','b2'),eda.Xor('a3','b3')))


write_circuit([comp1, comp2], circuit, ['comp1', 'comp2'])

## Equivalence check adder 1 and 2
circuit1 = ['comp1']
circuit2 = ['comp2']
out = 1
res = equivalence_check(circuit, circuit1, circuit2, out)

['comp1'] and ['comp2'] are UNSAT --> EQUIVALENT CIRCUITS


In [29]:
a, b, c = map(eda.exprvar, 'abc')
f1 = eda.Or(eda.And(~a, ~b, ~c), eda.And(~a, ~b, c), eda.And(a, ~b, c), eda.And(a, b, c), eda.And(a, b, ~c))

In [30]:
d = circuit2dimacs(f1)
write_dimacs("test5.cnf", d)

In [5]:
# a, b, c = map(exprvar, 'abc')
# f1 = Or(And(~a, ~b, ~c), And(~a, ~b, c), And(a, ~b, c), And(a, b, c), And(a, b, ~c))
# ex = f1.to_cnf()
# exp, dimacs = expr2dimacscnf(ex)
# print(dimacs)
# f2 = Or(And(~a, ~b, c), And(a, ~b, c), And(a, b, c), And(a, b, ~c))
# ex = f2.to_cnf()
# exp, dimacs = expr2dimacscnf(ex)
# print(dimacs)

In [ ]:
# vars_cnt, clauses, comments = read_dimacs('cnf_bench/test1.cnf')

# res, rev_st = dimacs.dimacs_to_pysmt(vars_cnt, clauses, comments)
# # varA = Symbol("A")
# f = Xor(res, res)

# sat_res = is_sat(f)
# # assert sat_res # SAT
# print("dimacs := %s is SAT? %s" % (res, sat_res))
# print("if UNSAT, given circuits are equal")
# # assert not sat_res # UNSAT